<a href="https://colab.research.google.com/github/dawit-melka/Amharic-Hate-speech-detection/blob/main/mBERT/Kaggle%20-%20Amharic_Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

dawitmelka_amharic_hate_speech_path = kagglehub.dataset_download('dawitmelka/amharic-hate-speech')
dawitmelka_amharic_hate_speech_dataset2_path = kagglehub.dataset_download('dawitmelka/amharic-hate-speech-dataset2')
dawitmelka_amharic_hate_speech_dataset_path = kagglehub.dataset_download('dawitmelka/amharic-hate-speech-dataset')

print('Data source import complete.')


<a href="https://colab.research.google.com/github/dawit-melka/Amharic-Hate-speech-detection/blob/main/mBERT/Amharic_Hate_Speech_detection_using_mBERT_(Trainer_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *Amharic Hate Speech Detection using fine-tuned mBERT*



### *Step 1: Install the necessary packages*

In [ ]:
#installing the transformers package
!pip install transformers

In [ ]:
#installing the datasets package
!pip install datasets

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
#install huggingface_hub package to interact with huggingface platform
!pip install huggingface_hub

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
!pip install wandb

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


### *Step 2: Import the necessay libraries from the installed packages*

In [ ]:
from datasets import Dataset
import datasets
from evaluate import load

In [ ]:
import numpy as np
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification
from google.colab import userdata

In [ ]:
from sklearn.model_selection import train_test_split

### *Step 3: Import the dataset to be used for Training the model*


The dataset used for this project is an Amharic dataset that was made available by Usmaelabdureman. It contains Amharic posts and comments retrieved from Facebook. It has 40,000 rows. The dataset can be accessed from [here](https://github.com/Usmaelabdureman/hate-speech-api/tree/main/data)

In [ ]:
#fetch the dataset from the drive
Labels=pd.read_csv('/kaggle/input/amharic-hate-speech/labels.txt',header=None)
Posts=pd.read_csv('/kaggle/input/amharic-hate-speech/posts.txt',header=None)

In [ ]:
def read_file(file_path):
    # Read with explicit handling of quotes and delimiters
    df = pd.read_csv(
        file_path,
        quotechar='"',
        skipinitialspace=True,
        on_bad_lines="skip",
        sep=","
    )
    if len(df.columns) == 1 and "Content,Label" in df.columns[0]:
        print(f"Splitting the single column...\n")
        df = df[df.columns[0]].str.split(",", n=1, expand=True)
    elif len(df.columns) != 2:
        raise ValueError(f"Unexpected number of columns in {file_path}.")
    df.columns = ["post", "labels"]
    df["post"] = df["post"].str.strip()
    df["labels"] = df["labels"].str.strip()
    df = df.dropna()
    print(f"Loaded {file_path} with shape: {df.shape}")
    print(f"{df.head()}\n")
    return df

In [ ]:
# Paths to files
train_path = '/kaggle/input/amharic-hate-speech-dataset/AMHSDataTrain.txt'
test_path = '/kaggle/input/amharic-hate-speech-dataset/AMHSDataTest.txt'

# Read datasets
train_df = read_file(train_path)
test_df = read_file(test_path)

# Combine datasets
# combined_df = pd.concat([train_df, test_df], ignore_index=True)

Splitting the single column...

Loaded /kaggle/input/amharic-hate-speech-dataset/AMHSDataTrain.txt with shape: (1594, 2)
                                                post labels
0  ሌቦች የአብይን ፎቶ ከጀርባህ ለጥፈህ ትለፈልፋለህ አብይ ላይ ያለንም እም...    ጥላቻ
1                  ህዝቤ አንድ ቀን ትግሬን ይበቀላል ጊዜ ዳኛው ቶሎ ና    ጥላቻ
2                                   ህዝብን የጨርስህ ጅብ ነህ    ጥላቻ
3                   ህገ መንግስታችን ይከበር ምድረ ጅቦች ሰው በላ ሁላ    ጥላቻ
4                                            ሆዳም ባንዳ    ጥላቻ

Splitting the single column...

Loaded /kaggle/input/amharic-hate-speech-dataset/AMHSDataTest.txt with shape: (396, 2)
                                                post labels
0  ዘረኛ ሰው ነው ስለሌላ ዘር ቀን ከለሊት ሲለፈልፍ የሚውለው  ዘረኛ መሆን...    ጥላቻ
1  ዘፈኑ ጥሩ ነው አንድ ሆዳም አማራ ግን የትግራይን የሽንት ጨርቅ በረጅም ...    ጥላቻ
2  ዛሬ ቅማላም ትግሬዎች በህዝባችን የተተፉና በገጀራ ዋጋቸውን የሚያገኙበት ...    ጥላቻ
3  ዝባዝንኬ አንዳንድ ኤፍ ኤም ሬድዮ ጣቢያዎች ላይ የሚተላለፉ ሸርሙጣ ሸርሙ...    ጥላቻ
4  የሀበሻ ምግብ ሂጄ ቅማላም ትግሬዎች ሲንጫጩ አበሸቁኝ እኛ በስንት ቀን አ...    ጥላቻ



### *Step 4: Preprocess the Dataset*

When the dataset was retrived, the labels and the post were in different files.


*   Hence, the first step in this phase is merging the files into one panda's dataframe.
*   Second step is Label encoding. Lable encoding is the process of converting the labels(classes) into numeric format to make it easier for the machine to understand it
*   Third step is dividing the dataset into training, validation and testing categories. The division ratio is 7:1:2 respectively.
*   Last step is to remove an unncessary columns from the main dataset and merging the all the categories into one main dataset



In [ ]:
train_df['labels'].unique()

array(['ጥላቻ', 'መልካም'], dtype=object)

In [ ]:
#naming the columns
Labels.column = ["labels"]
Posts.columns = ["post"]

In [ ]:
#encoding the classes into numerical data
Labels = Labels.replace(['Free', 'Free ','Hate'],[0,0,1])

In [ ]:
train_df['labels'] = train_df['labels'].replace(['መልካም','ጥላቻ'],[0, 1])

In [ ]:
unique_values = Labels['labels'].unique()
print(unique_values)

In [ ]:
#check the encoded label data
Labels.head(10)

,labels
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [ ]:
#check the Amharic data
Posts.head(1000)

,post
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...
995,መፈንቅለመንግስት ከሽፏልእንዴ የየክልሉ ህዝብ ምን ይጠብቃል ሆ ብሎ ሄዶ ...
996,ሰው በሩን እንደፈለገ መክፈት እንደፈለገ መዝጋት ይችላል የምን አውቅልሻ...
997,ክክክክክ የሱዳን ዜጋ ናችሁ እንዴ ባለ ነጠላ ጎጃሜ ሁላ
998,የምን ማለቃቀስ ነው ምድረ ፎካሪ ሁላ


In [ ]:
#merge the datasets
Frames = [Labels, Posts]
Merged = pd.concat(Frames, axis=1)


In [ ]:
Merged = train_df

In [ ]:
#preview of merged data
Merged

,post,labels
0,ሌቦች የአብይን ፎቶ ከጀርባህ ለጥፈህ ትለፈልፋለህ አብይ ላይ ያለንም እም...,1
1,ህዝቤ አንድ ቀን ትግሬን ይበቀላል ጊዜ ዳኛው ቶሎ ና,1
2,ህዝብን የጨርስህ ጅብ ነህ,1
3,ህገ መንግስታችን ይከበር ምድረ ጅቦች ሰው በላ ሁላ,1
4,ሆዳም ባንዳ,1
...,...,...
1595,አዳራሹ ሰፊ አጀንዳው ቁሞቀር በጠባብ ቤት ሆነን ብንሰፋ ምን ነበር,0
1596,አገሪቷን የሚበዘብዙት የትግራይ ሚሊሻ ጀነራል ብሎ እራሳቸውን ሹሞ አገር...,0
1597,አገሬ ኢትዮጵያ የማንም መጫወቻ ሆና ትቅር ያሳዝናል ሰው ለከቱን ሲያጣ,0
1598,አማራ በክልል ብቻ ታጥሮ ጠቦ የሚኖር ማህበረሰብ አይደለም እንደሌሎች ጠብ...,0


In [ ]:
# train_val_df, test_dataset = train_test_split(Merged, test_size=0.20, random_state=42)
# train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.10, random_state=42)
test_dataset = test_df
train_dataset, evaluation_dataset = train_test_split(Merged, test_size=0.10, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (1434, 2)
Validation dataset shape:  (160, 2)
Testing dataset shape:  (396, 2)


In [ ]:
#Divide the dataset into train and test categories
msk = np.random.rand(len(Merged)) < 0.8
train_dataset = Merged[msk]
test_dataset = Merged[~msk]

In [ ]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)

In [ ]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [ ]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
evaluation_dataset=Dataset.from_pandas(evaluation_dataset)

In [ ]:
#preview of the dataset after conversion
(test_dataset)

Dataset({
    features: ['labels', 'post', '__index_level_0__'],
    num_rows: 9090
})

In [ ]:
#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [ ]:
#combine the train and test dataset into one datset
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [ ]:
#preview of the dataset after merging
main_dataset

DatasetDict({
    train: Dataset({
        features: ['post', 'labels'],
        num_rows: 1434
    })
    test: Dataset({
        features: ['post', 'labels'],
        num_rows: 396
    })
    evaluate: Dataset({
        features: ['post', 'labels'],
        num_rows: 160
    })
})

In [ ]:
# training and testing data size
training_data_size = main_dataset['train'].num_rows
testing_data_size = main_dataset['test'].num_rows
evaluation_data_size = main_dataset['evaluate'].num_rows

### *Step 5: Tokenizing Dataset*

A Tokenizer is used to translate text into data that can be processed by the model. Models can only process numbers, so tokenizers need to convert our text inputs to numerical data.

In this case, the tokenizer used is an AutoTokenizer from the fine-tuned mBERT model made available by Hugging face [here](https://huggingface.co/Davlan/bert-base-multilingual-cased-finetuned-amharic)


In this phase, we have the following tasks:
* Load the tokenizer
* Create a tokenizer function that takes the dataset in batches and tokenize them using the tokenizer loaded from the model
* Call the tokenizer function on the whole dataset

In [ ]:
#loading a tokenizer from the pretrained model
tokenizer = AutoTokenizer.from_pretrained("DawitMelka/amharic-hate-speech")

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
#Have a tokenizer function that uses the tokenizer
def tokenize_function(data):
    return tokenizer(data["post"], padding="max_length", truncation=True)

In [ ]:
#Tokenize all the data using the mapping functionality
tokenized_datasets = main_dataset.map(tokenize_function)


Map:   0%|          | 0/1434 [00:00<?, ? examples/s]

Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [ ]:
#empty cache
torch.cuda.empty_cache()

### *Step 6: Prepare the tokenized Dataset*

In this phase, we do the following tasks:

* Remove unnecessary columns such as the "posts" column from the tokenized dataset as we no longer need them
* Change the format of the tokenized dataset into pytorch since we are using pytorch
* Load the dataset using DataLoader with the proper batch size
* Preview the features of the dataset to make sure everything is okay

In [ ]:
#remove the posts column as it is no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["post"])

In [ ]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [ ]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))
small_eval_dataset = tokenized_datasets["evaluate"].shuffle(seed=42).select(range(evaluation_data_size))

In [ ]:
# preview of the shuffeled and selected evaluation dataset
small_eval_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 160
})

In [ ]:
# preview of the shuffeled and selected training dataset
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1434
})

In [ ]:
# preview of the shuffeled and selected testing dataset
small_test_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 396
})

In [ ]:
#load the dataset using DataLoader
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)
test_dataloader = DataLoader(small_test_dataset, batch_size=4)

### *Step 7: Fine-tune the model*

This phase has the following steps:
* Load the model
* Specify the computing metric
* Specify the Training/fine-tuning arguments
* Load the Trainer class
* Fine-tune the model

**7.1 Load the model**<br>
We load the fine-tuned mBERT mode in this step

In [ ]:

# model = AutoModelForSequenceClassification.from_pretrained("indominyus/amharic-hate-speech-detection-mBERT", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("Davlan/", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/hate_speech_training_with_callbacks")
# tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/training_with_callbacks")

model = AutoModelForSequenceClassification.from_pretrained("DawitMelka/amharic-hate-speech")
tokenizer = AutoTokenizer.from_pretrained("DawitMelka/amharic-hate-speech")

config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

**7.2 Computing Metrics** <br>
In this stage, we load the computing metrics. The computing metrics used in this phase are the f1-score and the accuracy. These computing metrics are used during the validation and testing phase  

In [ ]:
import numpy as np

metric = load("f1","accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

**7.3 Specify the training arguments** <br>
This phase includes loading the training parameters and hyperparameters.
It also specifies the validation interval during the fine-tuning process.

In [ ]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from transformers import EarlyStoppingCallback, IntervalStrategy

In [ ]:
training_args = TrainingArguments(
   f"hate_speech_training_with_callbacks",
   eval_strategy = IntervalStrategy.STEPS,
   warmup_steps=500,
   save_steps=2000,
   eval_steps = 2000,
   save_total_limit = 3,
   learning_rate=1e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=10,
   weight_decay=0.01,
   push_to_hub=False,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True
   # resume_from_checkpoint=True
)

**7.4 Load the Trainer class**<br>
In the trainer class, early stopping strategy is called. Early Stopping is a an optimization technique used to reduce overfitting without compromising on model accuracy. It allows to specify an arbitrary large number of training epochs and stop training once the model performance stops improving on a hold out validation dataset. For this model, the early stopping patience used is 10 epoches.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
)

**7.5 Fine-tune the model** <br>
Fine-tuning process embbeds the validation within itself. After every 2000 steps of finetuning, the model is validated on the loaded computing metrics to modify the hyperparameters to make the model perform well

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
key = user_secrets.get_secret("wandb_key")

!wandb login $key
#wandb.init(project='amharic-hate-speech-detection', settings=wandb.Settings(init_timeout=120))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
wandb.init(project='amh-hate-speech-detection', settings=wandb.Settings(init_timeout=420))

In [ ]:
# trainer.train(resume_from_checkpoint=True)
trainer.train()

Step,Training Loss,Validation Loss,F1
2000,0.042300,0.982505,0.895028


TrainOutput(global_step=3590, training_loss=0.17444899002489606, metrics={'train_runtime': 920.8193, 'train_samples_per_second': 15.573, 'train_steps_per_second': 3.899, 'total_flos': 3773012533862400.0, 'train_loss': 0.17444899002489606, 'epoch': 10.0})

### *Step 8: Test the model*

In this stage the model is tested on the testing dataset. This dataset isn't seen by the model during the finetuning process.

In [ ]:
trainer.evaluate(small_test_dataset)

early stopping required metric_for_best_model, but did not find eval_f1 so early stopping is disabled


{'eval_runtime': 6.0758,
 'eval_samples_per_second': 65.177,
 'eval_steps_per_second': 16.294,
 'epoch': 10.0}

### *Step 9: Push the model to Huggingface Hub*

One of the aims of this research/project is to contribute to the IT community in the sector of NLP tasks on low-resourced languages. Hence, the final model was pushed and made publicly available on Huggingface. You can find the model on huggingface [here](https://huggingface.co/amengemeda/improved-amharic-hate-speech-detection-mBERT)


In [ ]:
from huggingface_hub import notebook_login, login

In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model_name = "dawitmelka/amharic-hate-speech"
model.save_pretrained("/kaggle/working/hate_speech_training_with_callbacks", save_config=True)
tokenizer.save_pretrained("/kaggle/working/hate_speech_training_with_callbacks")

!ls /kaggle/working/hate_speech_training_with_callbacks

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/hate_speech_training_with_callbacks")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/hate_speech_training_with_callbacks")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2535: UserWarning: `save_config` is deprecated and will be removed in v5 of Transformers. Use `is_main_process` instead.
  warnings.warn(


checkpoint-2000   config.json	     special_tokens_map.json  vocab.txt
checkpoint-90000  model.safetensors  tokenizer_config.json
checkpoint-90010  runs		     tokenizer.json


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
model_name = "DawitMelka/amharic-hate-speech"

login(token=userdata.get('HF_KEY'))

# # Push to HuggingFace Hub
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DawitMelka/amharic-hate-speech/commit/f3e461d03634b41d7f6cbfe377e68fcc3dc78b4f', commit_message='Upload tokenizer', commit_description='', oid='f3e461d03634b41d7f6cbfe377e68fcc3dc78b4f', pr_url=None, pr_revision=None, pr_num=None)

## **Test the model with different dataset**

In [ ]:
def read_file(file_path):
    # Read with explicit handling of quotes and delimiters
    df = pd.read_csv(
        file_path,
        quotechar='"',
        skipinitialspace=True,
        on_bad_lines="skip",
        sep=","
    )
    # Check the structure of the loaded DataFrame
    print(f"First few rows:\n{df.head()}\n")
    print(f"Loaded {file_path} with shape: {df.shape}\n")
    print(f"Columns: {df.columns}\n\n")
    # Split the single column if needed
    if len(df.columns) == 1 and "Content,Label" in df.columns[0]:
        print(f"Splitting the single column...\n")
        df = df[df.columns[0]].str.split(",", n=1, expand=True)
        df.columns = ["Content", "Label"]
        print(f"After splitting:\n{df.head()}\n")
        print(f"Loaded {file_path} with shape: {df.shape}")
    elif len(df.columns) != 2:
        raise ValueError(f"Unexpected number of columns in {file_path}.")
    return df


In [ ]:
# Truncate or split long texts
def process_text(text, tokenizer, max_length=512):
    # Tokenize and check length
    tokens = tokenizer.encode(text, add_special_tokens=True)

    # If tokens are too long, truncate
    if len(tokens) > max_length:
        tokens = tokens[:max_length]
        text = tokenizer.decode(tokens)

    return text

In [ ]:
# Paths to files
train_path = '/content/drive/MyDrive/Hate speech dataset/AMHSDataTrain.csv'
test_path = '/content/drive/MyDrive/Hate speech dataset/AMHSDataTest.csv'

# Read datasets
train_df = read_file(train_path)
test_df = read_file(test_path)

# Combine datasets
combined_df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
print(combined_df.head())
combined_df.shape

In [ ]:
# Load model and tokenizer
# model_name = "amengemeda/amharic-hate-speech-detection-mBERT"
model_name = "shuaibam/amharic-hate-speech-detection-mBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
# Create classification pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


In [ ]:
# Manual prediction function
def predict_hate_speech(texts, model, tokenizer, max_length=512):
    model.eval()
    predictions = []

    for text in texts:
        # Tokenize with truncation
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=max_length,
            padding=True
        )

        # Predict
        with torch.no_grad():
            outputs = model(**inputs)
            pred = torch.softmax(outputs.logits, dim=1)
            pred_label = torch.argmax(pred, dim=1).item()
            predictions.append('hate' if pred_label == 1 else 'free')

    return predictions

In [ ]:
# Predict
combined_df['Predicted_Label'] = predict_hate_speech(combined_df['Content'], model, tokenizer)

In [ ]:
# Analyze predictions
total_samples = len(combined_df)
correct_predictions = (combined_df['Label'] == combined_df['Predicted_Label']).sum()
accuracy = correct_predictions / total_samples * 100

# Confusion matrix
confusion = pd.crosstab(combined_df['Label'], combined_df['Predicted_Label'])

# False Positives and False Negatives
TP = confusion.loc['መልካም', 'መልካም']
FN = confusion.loc['መልካም', 'ጥላቻ']
FP = confusion.loc['ጥላቻ', 'መልካም']
TN = confusion.loc['ጥላቻ', 'ጥላቻ']
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

In [ ]:
# Detailed results
print("Total Samples:", total_samples)
print("Accuracy: {:.2f}%".format(accuracy))
print("Precision: {:.2f}%".format(precision))
print("Recall: {:.2f}%".format(recall))
print("F1 Score: {:.2f}%".format(f1_score))

print(f"\nTrue Positives (TP): {TP}")
print(f"False Negatives (FN): {FN}")
print(f"False Positives (FP): {FP}")
print(f"True Negatives (TN): {TN}\n")

print("\nConfusion Matrix:")
print(confusion)

In [ ]:
# False Positives and False Negatives
false_positives = combined_df[(combined_df['Label'] == 'መልካም') & (combined_df['Predicted_Label'] == 'ጥላቻ')]
false_negatives = combined_df[(combined_df['Label'] == 'ጥላቻ') & (combined_df['Predicted_Label'] == 'መልካም')]

# Save results

false_positives.to_csv('/content/drive/MyDrive/Hate speech dataset/false_positives_full.csv', index=False)
false_negatives.to_csv('/content/drive/MyDrive/Hate speech dataset/false_negatives_full.csv', index=False)
